# 🎯 Quick Experiment Runner

This notebook lets you run **individual experiments** with customizable settings.

Just select which experiment you want to run and adjust the parameters!

---

## 1️⃣ Setup (Run First!)

In [ ]:
# ============================================================
# SETUP - Run this cell first!
# ============================================================

import subprocess
import sys
import os
from pathlib import Path

# Find project paths
notebook_dir = Path(os.getcwd())
if 'notebooks' in str(notebook_dir):
    project_root = notebook_dir.parent
else:
    project_root = notebook_dir

experiments_dir = project_root / 'experiments'

print("✅ Setup Complete!")
print(f"📁 Project: {project_root}")
print(f"📁 Experiments: {experiments_dir}")

# Check GPU
import torch
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
else:
    print("💻 CPU mode")

---

## 2️⃣ Configure Your Experiment

Modify the settings below, then run the experiment cell.

In [ ]:
# ============================================================
# CONFIGURATION - Modify these!
# ============================================================

# Which experiment to run? (1, 2, 3, 4, or 5)
EXPERIMENT_NUMBER = 1

# Common settings
CLASS_NAME = "Tomato"        # Filter to specific crop ("Tomato", "Apple", "Grape", etc.)
EPOCHS = 5                    # Number of training epochs
BATCH_SIZE = 16               # Batch size (reduce to 8 if memory error)

# Active learning settings (for experiments 4 & 5)
BUDGET_PER_ROUND = 50         # Samples to label per round
NUM_ROUNDS = 4                # Number of rounds

# CutMix settings (for experiment 3)
CUTMIX_PROB = 0.5             # Probability of applying CutMix

print(f"\n📋 Configuration:")
print(f"   Experiment: {EXPERIMENT_NUMBER}")
print(f"   Class: {CLASS_NAME}")
print(f"   Epochs: {EPOCHS}")
print(f"   Batch size: {BATCH_SIZE}")

---

## 3️⃣ Run the Experiment

This will execute the selected experiment with your settings.

In [ ]:
# ============================================================
# RUN EXPERIMENT
# ============================================================

experiment_scripts = {
    1: '01_baseline_gap.py',
    2: '02_passive_aug.py',
    3: '03_cutmix.py',
    4: '04_active_learning.py',
    5: '05_hybrid_warmstart.py'
}

experiment_names = {
    1: 'Baseline Generalization Gap',
    2: 'Passive Augmentation',
    3: 'CutMix Augmentation',
    4: 'Active Learning Comparison',
    5: 'Hybrid Warm-Start (Our Method)'
}

if EXPERIMENT_NUMBER not in experiment_scripts:
    print(f"❌ Invalid experiment number: {EXPERIMENT_NUMBER}")
    print("   Choose 1, 2, 3, 4, or 5")
else:
    script = experiments_dir / experiment_scripts[EXPERIMENT_NUMBER]
    name = experiment_names[EXPERIMENT_NUMBER]
    
    print("="*60)
    print(f"🧪 Running Experiment {EXPERIMENT_NUMBER}: {name}")
    print("="*60)
    
    # Build command
    cmd = [
        sys.executable, str(script),
        '--class-name', CLASS_NAME,
        '--epochs', str(EPOCHS),
        '--batch-size', str(BATCH_SIZE)
    ]
    
    # Add experiment-specific args
    if EXPERIMENT_NUMBER == 3:
        cmd.extend(['--cutmix-prob', str(CUTMIX_PROB)])
    elif EXPERIMENT_NUMBER in [4, 5]:
        cmd.extend([
            '--budget-per-round', str(BUDGET_PER_ROUND),
            '--num-rounds', str(NUM_ROUNDS)
        ])
    
    print(f"\n📌 Command: {' '.join(cmd)}\n")
    print("-"*60)
    
    # Run
    result = subprocess.run(cmd, cwd=str(experiments_dir))
    
    print("-"*60)
    if result.returncode == 0:
        print("\n✅ Experiment completed successfully!")
    else:
        print(f"\n❌ Experiment failed with code {result.returncode}")

---

## 📋 Experiment Descriptions

| # | Name | Description | Run Time |
|---|------|-------------|----------|
| 1 | **Baseline Gap** | Train on Lab, test on Field. Shows the problem. | ~5 min |
| 2 | **Passive Augmentation** | Strong augmentation to simulate field conditions | ~5 min |
| 3 | **CutMix** | Patch mixing regularization | ~10 min |
| 4 | **Active Learning** | Compare Random vs Entropy sampling | ~15 min |
| 5 | **Hybrid (Ours)** | Our proposed warm-start method | ~20 min |

---

## 💡 Tips

- **Memory error?** Reduce `BATCH_SIZE` to 8 or 4
- **Better results?** Increase `EPOCHS` to 10-15
- **Faster testing?** Use fewer `NUM_ROUNDS` (e.g., 2)
- **Different crops?** Try `CLASS_NAME = "Apple"` or `"Grape"`